In [1]:
import uuid 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SilverVendas") \
    .config("spark.extraListeners", "io.openlineage.spark.agent.OpenLineageSparkListener") \
    .config("spark.openlineage.columnLineage.datasetLineageEnabled", "true") \
    .config("spark.openlineage.transport.type", "http") \
    .config("spark.openlineage.transport.url", "http://api:5000") \
    .config("spark.openlineage.namespace", "spark_integration") \
    .config("spark.openlineage.parentJobName", "customer-job") \
    .config("spark.openlineage.parentRunId", uuid.uuid4().__str__()) \
    .getOrCreate()

25/06/19 18:12:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
delta_path_vendas = "s3a://bronze/vendas/"
delta_path_pessoas = "s3a://bronze/pessoas/"

# Ler os dados
df_vendas = spark.read.format("delta").load(delta_path_vendas)
df_pessoas = spark.read.format("delta").load(delta_path_pessoas)

25/06/19 18:12:20 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [3]:
df_vendas.show()

25/06/19 18:12:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 10:>                                                         (0 + 1) / 1]

+--------+----------+-----+--------------------+-----------+
|order_id| food_item|price|          order_time|id_customer|
+--------+----------+-----+--------------------+-----------+
|       1|     Sushi|32.05|2025-06-19 15:13:...|          4|
|       2|   Lasanha|34.07|2025-06-19 04:55:...|          1|
|       3|    Salada|28.15|2025-06-18 19:33:...|          2|
|       4|Hamburguer| 79.4|2025-06-18 20:45:...|          2|
|       5|     Pizza|43.74|2025-06-19 15:40:...|          1|
|       6|     Pizza|76.93|2025-06-18 23:20:...|          4|
|       7|     Sushi|63.18|2025-06-19 07:16:...|          5|
|       8|     Pizza|17.07|2025-06-19 02:37:...|          1|
|       9|    Salada| 25.8|2025-06-19 02:09:...|          2|
|      10|   Lasanha|65.81|2025-06-19 13:43:...|          3|
|      11|    Salada|33.66|2025-06-19 04:37:...|          5|
|      12|    Salada|53.45|2025-06-18 21:25:...|          3|
|      13|     Sushi| 36.1|2025-06-18 22:17:...|          4|
|      14|     Pizza|46.

In [4]:
df_pessoas.show()

25/06/19 18:12:34 ERROR ContextFactory: Query execution is null: can't emit event for executionId 8
25/06/19 18:12:34 ERROR ContextFactory: Query execution is null: can't emit event for executionId 8
25/06/19 18:12:34 ERROR ContextFactory: Query execution is null: can't emit event for executionId 9
[Stage 16:=====================================================>  (48 + 2) / 50]

+-----------+-------------+
|id_customer|customer_name|
+-----------+-------------+
|          1|          Ana|
|          2|       Carlos|
|          3|         João|
|          4|        Lucas|
|          5|        Maria|
+-----------+-------------+



In [5]:
from pyspark.sql.functions import col, current_date

df_joined = df_vendas.join(
    df_pessoas,
    on="id_customer",
    how="left"
).withColumnRenamed("order_id", "nk_order") \
.withColumnRenamed("food_item", "item") \
.withColumnRenamed("price", "price") \
.withColumnRenamed("order_time", "order_time") \
.withColumn("etl_date", current_date())  # Adiciona a data do processamento

In [6]:
df_joined.show()

25/06/19 18:12:41 ERROR ContextFactory: Query execution is null: can't emit event for executionId 13
25/06/19 18:12:41 ERROR ContextFactory: Query execution is null: can't emit event for executionId 13
[Stage 26:>                                                         (0 + 1) / 1]

+-----------+--------+----------+-----+--------------------+-------------+----------+
|id_customer|nk_order|      item|price|          order_time|customer_name|  etl_date|
+-----------+--------+----------+-----+--------------------+-------------+----------+
|          4|       1|     Sushi|32.05|2025-06-19 15:13:...|        Lucas|2025-06-19|
|          1|       2|   Lasanha|34.07|2025-06-19 04:55:...|          Ana|2025-06-19|
|          2|       3|    Salada|28.15|2025-06-18 19:33:...|       Carlos|2025-06-19|
|          2|       4|Hamburguer| 79.4|2025-06-18 20:45:...|       Carlos|2025-06-19|
|          1|       5|     Pizza|43.74|2025-06-19 15:40:...|          Ana|2025-06-19|
|          4|       6|     Pizza|76.93|2025-06-18 23:20:...|        Lucas|2025-06-19|
|          5|       7|     Sushi|63.18|2025-06-19 07:16:...|        Maria|2025-06-19|
|          1|       8|     Pizza|17.07|2025-06-19 02:37:...|          Ana|2025-06-19|
|          2|       9|    Salada| 25.8|2025-06-19 02:0

In [7]:
# Caminho para o bucket MinIO (formato S3)
delta_path = "s3a://silver/fat_vendas"

# Escrever como tabela Delta
df_joined.write.format("delta").mode("overwrite").save(delta_path)

25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for executionId 16
25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for executionId 16
25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for executionId 18
25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for executionId 18
25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for executionId 19
25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for executionId 19
25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for executionId 20
25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for executionId 20
25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for executionId 21
25/06/19 18:12:46 ERROR ContextFactory: Query execution is null: can't emit event for execu

In [8]:
spark.stop()